In [2]:
import sys, os
import re
from collections import defaultdict
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
from matplotlib.patches import Rectangle, Polygon
import seaborn as sns
sns.set_style('white')

import geneinfo as gi
gi.email('ariadna.saez@alum.esci.upf.edu')


# for use on the cluster
%env ftp_proxy http://proxy-default:3128

env: ftp_proxy=http://proxy-default:3128


In [3]:
help(gi)

Help on package geneinfo:

NAME
    geneinfo

PACKAGE CONTENTS
    intervals

CLASSES
    builtins.Exception(builtins.BaseException)
        NotFound
    builtins.object
        WrSubObo
        nice
    goatools.go_enrichment.GOEnrichmentRecord(builtins.object)
        My_GOEnrichemntRecord
    
    class My_GOEnrichemntRecord(goatools.go_enrichment.GOEnrichmentRecord)
     |  My_GOEnrichemntRecord(goid, **kwargs)
     |  
     |  Represents one result (from a single GOTerm) in the GOEnrichmentStudy
     |  
     |  Method resolution order:
     |      My_GOEnrichemntRecord
     |      goatools.go_enrichment.GOEnrichmentRecord
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __str__(self)
     |      Return str(self).
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from goatools.go_enrichment.GOEnrichmentRecord:
     |  
     |  __init__(self, goid, **kwargs)
     |      Initialize sel

In [2]:
## RELATE ##

In [23]:
## AFRICANS ##
import pandas as pd
import os

# Define the threshold
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/LWK/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/ESN/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/MSL/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/GWD/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/YRI/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = file_path.split('/')[-4]  # Get the third element from the end when splitting by '/'
    
    # Read data with '\s+' separator and make the values in "when_mutation_has_freq2" column positive
    data = pd.read_csv(file_path, sep='\s+')
    data['when_mutation_has_freq2'] = data['when_mutation_has_freq2'].abs()
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['when_mutation_has_freq2'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
            # Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

    # Print gene regions for the current population
    print(f"Gene regions for {population_name}:")
    for region in population_gene_regions:
        print(region)
    print()


Gene regions for LWK:
('PTCHD1', 23334395, 23404374)
('CASK', 41514933, 41665852)
('FAM120C', 54068323, 54183254)
('LOC105377212', 63237521, 63277646)
('LINC01278', 63426557, 63561095)
('AMMECR1', 110194185, 110318085)
('RTL4', 112083012, 112457514)
('ENOX2', 130622324, 130720491)
('RAP2C-AS1', 132218506, 132432811)
('MAMLD1', 150361571, 150514173)
('ZNF185', 152898066, 152973474)

Gene regions for ESN:
('CDKL5', 18425607, 18640196)
('CDKL5', 18425607, 18640196)
('CDKL5', 18425607, 18640196)
('CDKL5', 18425607, 18640196)
('PTCHD1-AS', 22193004, 23293146)
('CASK', 41514933, 41923034)
('CASK', 41514933, 41923034)
('LOC124905191', 53361522, 53370780)
('RTL4', 112083012, 112457514)
('TENM1', 124375902, 124963817)
('IGSF1', 131273505, 131289306)
('G6PD', 154531389, 154547018)

Gene regions for MSL:
('PTCHD1-AS', 22193004, 23293146)

Gene regions for GWD:
('ARSL', 2934520, 2964341)
('BCOR', 40051245, 40177277)
('BCOR', 40051245, 40177277)
('HUWE1', 53532095, 53680089)
('ACSL4', 109641334, 10

In [25]:
## PUERTO RICANS ##
import pandas as pd
import os

# Define the threshold
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/PUR/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = file_path.split('/')[-4]  # Get the third element from the end when splitting by '/'
    
    # Read data with '\s+' separator and make the values in "when_mutation_has_freq2" column positive
    data = pd.read_csv(file_path, sep='\s+')
    data['when_mutation_has_freq2'] = data['when_mutation_has_freq2'].abs()
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['when_mutation_has_freq2'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
            # Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

    # Print gene regions for the current population
    print(f"Gene regions for {population_name}:")
    for region in population_gene_regions:
        print(region)
    print()


Gene regions for PUR:
('ARSL', 2934520, 2964341)
('SHROOM2', 9786428, 9949443)
('WWC3', 10015253, 10144474)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10677739)
('MID1', 10445309, 10677739)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 108

In [26]:
## EUROPEANS ##
import pandas as pd
import os

# Define the threshold
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/GBR/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/FIN/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/IBS/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/TSI/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = file_path.split('/')[-4]  # Get the third element from the end when splitting by '/'
    
    # Read data with '\s+' separator and make the values in "when_mutation_has_freq2" column positive
    data = pd.read_csv(file_path, sep='\s+')
    data['when_mutation_has_freq2'] = data['when_mutation_has_freq2'].abs()
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['when_mutation_has_freq2'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
            # Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

    # Print gene regions for the current population
    print(f"Gene regions for {population_name}:")
    for region in population_gene_regions:
        print(region)
    print()


Gene regions for GBR:
('PTCHD1-AS', 22193004, 23293146)
('PTCHD1-AS', 22193004, 23293146)

Gene regions for FIN:
('IL1RAPL1', 28587445, 29956718)
('DMD', 31119221, 31266954)
('DMD', 31119221, 33211549)
('XPNPEP2', 129738978, 129769536)

Gene regions for IBS:
('PRKX', 3604339, 3713649)
('PRKX', 3604339, 3713649)
('PRKX', 3604339, 3713649)
('PRKX', 3604339, 3713649)
('PRKX', 3604339, 3713649)
('NHS', 17375199, 17735994)
('PHKA1', 72578813, 72714306)
('MIR325HG', 76657797, 77014532)
('TENM1', 124375902, 124963817)
('TMLHE', 155489010, 155536365)

Gene regions for TSI:
('PRKX', 3604339, 3713649)
('LOC112268307', 64205945, 64312562)



In [27]:
## ASIANS ##
import pandas as pd
import os

# Define the threshold
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/CDX/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/CHS/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/CHB/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/JPT/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele',
    '/home/ari/ari-intern/people/ari/ariadna-intern/steps/KHV/relate/run_relate/1000g_ppl_phased_haplotypes_selection.sele'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = file_path.split('/')[-4]  # Get the third element from the end when splitting by '/'
    
    # Read data with '\s+' separator and make the values in "when_mutation_has_freq2" column positive
    data = pd.read_csv(file_path, sep='\s+')
    data['when_mutation_has_freq2'] = data['when_mutation_has_freq2'].abs()
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['when_mutation_has_freq2'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
            # Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

    # Print gene regions for the current population
    print(f"Gene regions for {population_name}:")
    for region in population_gene_regions:
        print(region)
    print()


Gene regions for CDX:
('WWC3', 10015253, 10144474)
('FRMPD4', 11822438, 12724523)
('FRMPD4', 11822438, 12724523)
('DMD', 31119221, 33211549)
('IL1RAPL2', 104566198, 105767829)

Gene regions for CHS:
('PRKX', 3604339, 3713649)
('CLCN4', 10156974, 10237660)
('LOC101928359', 122422005, 122478081)

Gene regions for CHB:
('PRKX', 3604339, 3713649)
('PRKX', 3604339, 3713649)
('DMD', 31119221, 33211549)
('PASD1', 151563674, 151676739)

Gene regions for JPT:
('CLCN5', 49922595, 50042541)

Gene regions for KHV:
('PASD1', 151563674, 151676739)
('TMEM187', 153972753, 153983194)



In [ ]:
## COMBINED ##

In [2]:
## AFRICANS##

# Define the threshold
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_LWK.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_ESN.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_GWD.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_MSL.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_YRI.csv'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read data
    data = pd.read_csv(file_path)
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['combined_p_values'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
# Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

# Print gene regions for all populations
for population, regions in gene_regions.items():
    print(f"Gene regions for {population}:")
    for region in regions:
        print(region)
    print()


print()
print()

# Set to store gene symbols for which information has already been retrieved
retrieved_symbols = set()

# Run gi.gene_info() for each unique gene symbol
for gene_symbol in unique_genes:
    # Check if gene information has already been retrieved for this symbol
    if gene_symbol not in retrieved_symbols:
        gene_info = gi.gene_info(gene_symbol)
        if gene_info is not None:
            print(f"Gene info for {gene_symbol}: {gene_info}")


Gene regions for combined_LWK:
('CASK', 41514933, 41665852)
('RTL4', 112083012, 112457514)

Gene regions for combined_ESN:
('CASK', 41514933, 41923034)
('RTL4', 112083012, 112457514)

Gene regions for combined_GWD:
('PAK3', 110944396, 111217494)

Gene regions for combined_MSL:
('PTCHD1-AS', 22193004, 23293146)

Gene regions for combined_YRI:
('PRKX', 3604339, 3713649)
('TMEM164', 110002368, 110177788)
('TMEM164', 110002368, 110177788)
('TMEM164', 110002368, 110177788)
('RAB33A', 130110622, 130184870)
('RAB33A', 130110622, 130184870)





**Symbol:** **_PTCHD1-AS_**   
*PTCHD1 antisense RNA (head to head)*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PTCHD1-AS)  


 ----

**Symbol:** **_PAK3_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** ARA, MRX30, MRX47, OPHN3, PAK-3, PAK3beta, XLID30, bPAK, beta-PAK  
*p21 (RAC1) activated kinase 3*  
**Summary:** The protein encoded by this gene is a serine-threonine kinase and forms an activated complex with GTP-bound RAS-like (P21), CDC2 and RAC1. This protein may be necessary for dendritic development and for the rapid cytoskeletal reorganization in dendritic spines associated with synaptic plasticity. Defects in this gene are the cause of a non-syndromic form of X-linked intellectual disability. Alternatively spliced transcript variants encoding different isoforms have been identified. [provided by RefSeq, Jul 2017].  
**Genomic position:** X:110944285-111227361 (hg38), X:110187513-110470589 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PAK3)  


 ----

**Symbol:** **_RAB33A_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** RabS10  
*RAB33A, member RAS oncogene family*  
**Summary:** The protein encoded by this gene belongs to the small GTPase superfamily, Rab family. It is GTP-binding protein and may be involved in vesicle transport. [provided by RefSeq, Jul 2008].  
**Genomic position:** X:130171962-130184870 (hg38), X:129305623-129318844 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=RAB33A)  


 ----

**Symbol:** **_PRKX_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** PKX1  
*protein kinase cAMP-dependent X-linked catalytic subunit*  
**Summary:** This gene encodes a serine threonine protein kinase that has similarity to the catalytic subunit of cyclic AMP dependent protein kinases. The encoded protein is developmentally regulated and may be involved in renal epithelial morphogenesis. This protein may also be involved in macrophage and granulocyte maturation. Abnormal recombination between this gene and a related pseudogene on chromosome Y is a frequent cause of sex reversal disorder in XX males and XY females. Pseudogenes of this gene are found on chromosomes X, 15 and Y. [provided by RefSeq, Feb 2010].  
**Genomic position:** X:3604340-3713649 (hg38), X:3522411-3631649 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PRKX)  


 ----

**Symbol:** **_TMEM164_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** bB360B22.3  
*transmembrane protein 164*  
**Summary:** Predicted to be integral component of membrane. [provided by Alliance of Genome Resources, Apr 2022]  
**Genomic position:** X:110002631-110182734 (hg38), X:109245859-109425962 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TMEM164)  


 ----

**Symbol:** **_RTL4_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** Mar4, Mart4, SIRH11, ZCCHC16  
*retrotransposon Gag like 4*  
**Summary:** Predicted to enable nucleic acid binding activity and zinc ion binding activity. Predicted to act upstream of or within cognition and norepinephrine metabolic process. [provided by Alliance of Genome Resources, Apr 2022]  
**Genomic position:** X:112083016-112457514 (hg38), X:111697727-111700473 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=RTL4)  


 ----

**Symbol:** **_CASK_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** CAGH39, CAMGUK, CMG, FGS4, LIN2, MICPCH, MRXSNA, TNRC8, hCASK  
*calcium/calmodulin dependent serine protein kinase*  
**Summary:** This gene encodes a calcium/calmodulin-dependent serine protein kinase. The encoded protein is a MAGUK (membrane-associated guanylate kinase) protein family member. These proteins are scaffold proteins and the encoded protein is located at synapses in the brain. Mutations in this gene are associated with FG syndrome 4, intellectual disability and microcephaly with pontine and cerebellar hypoplasia, and a form of X-linked intellectual disability. Multiple transcript variants encoding different isoforms have been found for this gene. [provided by RefSeq, Jul 2017].  
**Genomic position:** X:41514934-41923554 (hg38), X:41374187-41782716 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=CASK)  


 ----

In [3]:
## PUR ##

threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_PUR.csv'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read data
    data = pd.read_csv(file_path)
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['combined_p_values'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
# Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

# Print gene regions for all populations
for population, regions in gene_regions.items():
    print(f"Gene regions for {population}:")
    for region in regions:
        print(region)
    print()


print()
print()

# Set to store gene symbols for which information has already been retrieved
retrieved_symbols = set()

# Run gi.gene_info() for each unique gene symbol
for gene_symbol in unique_genes:
    # Check if gene information has already been retrieved for this symbol
    if gene_symbol not in retrieved_symbols:
        gene_info = gi.gene_info(gene_symbol)
        if gene_info is not None:
            print(f"Gene info for {gene_symbol}: {gene_info}")


Gene regions for combined_PUR:
('WWC3', 10015253, 10144474)
('WWC3', 10015253, 10144474)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10576917)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10620585)
('MID1', 10445309, 10677739)
('MID1', 10445309, 10677739)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('MID1', 10445309, 10833683)
('PHKA2', 18892297, 18938716)
('PHKA2', 18892297, 18938716)
('MAP3K15', 19360058, 19515508)
('PTCHD1-AS', 22193004, 23293146)
('TSPAN7', 38561541, 38688918)
('CLCN5', 49922595, 50042541)
('SHROOM4', 50575533, 50814194)
('SHROOM4', 50575533, 50814194)
('SHROOM4', 50575533, 50814194)
('APEX2', 55000362, 55009057)
('HMGN5', 81113698, 81201913)
('LOC107985713', 88494129, 88613490)
('LOC1079

**Symbol:** **_PTCHD1-AS_**   
*PTCHD1 antisense RNA (head to head)*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PTCHD1-AS)  


 ----

**Symbol:** **_LINC00850_** (ncRNA) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** KUCG1  
*long intergenic non-protein coding RNA 850*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=LINC00850)  


 ----

**Symbol:** **_MID1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** BBBG1, FXY, GBBB, GBBB1, MIDIN, OGS1, OS, OSX, RNF59, TRIM18, XPRF, ZNFXY  
*midline 1*  
**Summary:** The protein encoded by this gene is a member of the tripartite motif (TRIM) family, also known as the 'RING-B box-coiled coil' (RBCC) subgroup of RING finger proteins. The TRIM motif includes three zinc-binding domains, a RING, a B-box type 1 and a B-box type 2, and a coiled-coil region. This protein forms homodimers which associate with microtubules in the cytoplasm. The protein is likely involved in the formation of multiprotein structures acting as anchor points to microtubules. Mutations in this gene have been associated with the X-linked form of Opitz syndrome, which is characterized by midline abnormalities such as cleft lip, laryngeal cleft, heart defects, hypospadias, and agenesis of the corpus callosum. This gene was also the first example of a gene subject to X inactivation in human while escaping it in mouse. Alternative promoter use, alternative splicing and alternative polyadenylation result in multiple transcript variants that have different tissue specificities. [provided by RefSeq, Dec 2016].  
**Genomic position:** X:10445310-10833654 (hg38), X:10413350-10851773 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=MID1)  


 ----

**Symbol:** **_TSPAN7_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** A15, CCG-B7, CD231, DXS1692E, MRX58, MXS1, TALLA-1, TM4SF2, TM4SF2b, XLID58  
*tetraspanin 7*  
**Summary:** The protein encoded by this gene is a member of the transmembrane 4 superfamily, also known as the tetraspanin family. Most of these members are cell-surface proteins that are characterized by the presence of four hydrophobic domains. The proteins mediate signal transduction events that play a role in the regulation of cell development, activation, growth and motility. This encoded protein is a cell surface glycoprotein and may have a role in the control of neurite outgrowth. It is known to complex with integrins. This gene is associated with X-linked cognitive disability and neuropsychiatric diseases such as Huntington's chorea, fragile X syndrome and myotonic dystrophy. [provided by RefSeq, Jul 2008].  
**Genomic position:** X:38561542-38688920 (hg38), X:38420623-38548169 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TSPAN7)  


 ----

...

**Symbol:** **_MAGEA3-DT_**   
*MAGEA3 divergent transcript*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=MAGEA3-DT)  


 ----

**Symbol:** **_WWC3_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** BM042  
*WWC family member 3*  
**Summary:** This gene encodes a member of the WWC family of proteins, which also includes the WWC1 (KIBRA) gene product and the WWC2 gene product. The protein encoded by this gene includes a C2 domain, which is known to mediate homodimerization in the related WWC1 gene product. [provided by RefSeq, Sep 2011].  
**Genomic position:** X:10015254-10144474 (hg38), X:9983602-10112518 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=WWC3)  


 ----

**Symbol:** **_PHKA2_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** GSD9A, PHK, PYK, PYKL, XLG, XLG2  
*phosphorylase kinase regulatory subunit alpha 2*  
**Summary:** Phosphorylase kinase is a polymer of 16 subunits, four each of alpha, beta, gamma and delta. The alpha subunit includes the skeletal muscle and hepatic isoforms, and the hepatic isoform is encoded by this gene. The beta subunit is the same in both the muscle and hepatic isoforms, and encoded by one gene. The gamma subunit also includes the skeletal muscle and hepatic isoforms, which are encoded by two different genes. The delta subunit is a calmodulin and can be encoded by three different genes. The gamma subunits contain the active site of the enzyme, whereas the alpha and beta subunits have regulatory functions controlled by phosphorylation. The delta subunit mediates the dependence of the enzyme on calcium concentration. Mutations in this gene cause glycogen storage disease type 9A, also known as X-linked liver glycogenosis. Alternatively spliced transcript variants have been reported, but the full-length nature of these variants has not been determined.[provided by RefSeq, Feb 2010].  
**Genomic position:** X:18892298-18984114 (hg38), X:18910418-19002716 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PHKA2)  


 ----

**Symbol:** **_LOC107985713_** (ncRNA)  
*uncharacterized LOC107985713*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC107985713)  


 ----

**Symbol:** **_RTL4_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** Mar4, Mart4, SIRH11, ZCCHC16  
*retrotransposon Gag like 4*  
**Summary:** Predicted to enable nucleic acid binding activity and zinc ion binding activity. Predicted to act upstream of or within cognition and norepinephrine metabolic process. [provided by Alliance of Genome Resources, Apr 2022]  
**Genomic position:** X:112083016-112457514 (hg38), X:111697727-111700473 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=RTL4)  


 ----

In [4]:
## EUROPEANS ##
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_GBR.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_FIN.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_IBS.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_TSI.csv'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read data
    data = pd.read_csv(file_path)
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['combined_p_values'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
# Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

# Print gene regions for all populations
for population, regions in gene_regions.items():
    print(f"Gene regions for {population}:")
    for region in regions:
        print(region)
    print()


print()
print()

# Set to store gene symbols for which information has already been retrieved
retrieved_symbols = set()

# Run gi.gene_info() for each unique gene symbol
for gene_symbol in unique_genes:
    # Check if gene information has already been retrieved for this symbol
    if gene_symbol not in retrieved_symbols:
        gene_info = gi.gene_info(gene_symbol)
        if gene_info is not None:
            print(f"Gene info for {gene_symbol}: {gene_info}")

Gene regions for combined_GBR:

Gene regions for combined_FIN:

Gene regions for combined_IBS:
('PRKX', 3604339, 3713649)
('TENM1', 124375902, 124963817)

Gene regions for combined_TSI:
('LOC112268307', 64205945, 64312562)





**Symbol:** **_LOC112268307_** (protein-coding)  
*uncharacterized LOC112268307*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC112268307)  


 ----

**Symbol:** **_TENM1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** ODZ1, ODZ3, TEN-M1, TEN1, TNM, TNM1, ten-1  
*teneurin transmembrane protein 1*  
**Summary:** The protein encoded by this gene belongs to the tenascin family and teneurin subfamily. It is expressed in the neurons and may function as a cellular signal transducer. Several alternatively spliced transcript variants encoding different isoforms have been found for this gene. [provided by RefSeq, Sep 2009].  
**Genomic position:** X:124375903-125204312 (hg38), X:123509753-124097666 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TENM1)  


 ----

**Symbol:** **_PRKX_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** PKX1  
*protein kinase cAMP-dependent X-linked catalytic subunit*  
**Summary:** This gene encodes a serine threonine protein kinase that has similarity to the catalytic subunit of cyclic AMP dependent protein kinases. The encoded protein is developmentally regulated and may be involved in renal epithelial morphogenesis. This protein may also be involved in macrophage and granulocyte maturation. Abnormal recombination between this gene and a related pseudogene on chromosome Y is a frequent cause of sex reversal disorder in XX males and XY females. Pseudogenes of this gene are found on chromosomes X, 15 and Y. [provided by RefSeq, Feb 2010].  
**Genomic position:** X:3604340-3713649 (hg38), X:3522411-3631649 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PRKX)  


 ----

In [5]:
## ASIANS ##
threshold = 6

# Function to define window to search nearby genes
def get_window_range(position, window_size):
    x_min = position - window_size // 2
    x_max = position + window_size // 2
    return x_min, x_max

# List of file paths
file_paths = [
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_CDX.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_CHS.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_CHB.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_KHV.csv',
    '/home/ari/ari-intern/people/ari/ariadna-intern/results/combined_JPT.csv'
]

# Dictionary to store gene regions for each population
gene_regions = {}

# Set to store unique gene symbols
unique_genes = set()

# Iterate over file paths
for file_path in file_paths:
    # Extract population name from the filename
    population_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read data
    data = pd.read_csv(file_path)
    
    # Add 'population' column
    data['population'] = population_name
    
    # Filter data based on threshold
    filtered_data = data[data['combined_p_values'] > threshold]
    
    # Initialize a list to store gene regions for the current population
    population_gene_regions = []
    
    # Iterate over rows and calculate gene regions
    for index, row in filtered_data.iterrows():
        genomic_position = row['pos']
        window_size = 1000  # Set window size !!!!!!! 500 bp on each SNP side
        x_min, x_max = get_window_range(genomic_position, window_size)
        
        # Call gi.get_genes_region() for the current region
        gene_region = gi.get_genes_region('chrX', x_min, x_max, assembly='hg38')
        
        # Extract gene symbol, start position, and end position from the result
        if gene_region:  # Check if gene_region is not empty
            gene_symbol = gene_region[0][0]
            start_position = gene_region[0][1]
            end_position = gene_region[0][2]
            
            # Append the extracted information to the list of gene regions for the current population
            population_gene_regions.append((gene_symbol, start_position, end_position))
            
# Add gene symbol to set of unique genes
            unique_genes.add(gene_symbol)
    
    # Store the list of gene regions for the current population in the dictionary
    gene_regions[population_name] = population_gene_regions

# Print gene regions for all populations
for population, regions in gene_regions.items():
    print(f"Gene regions for {population}:")
    for region in regions:
        print(region)
    print()


print()
print()

# Set to store gene symbols for which information has already been retrieved
retrieved_symbols = set()

# Run gi.gene_info() for each unique gene symbol
for gene_symbol in unique_genes:
    # Check if gene information has already been retrieved for this symbol
    if gene_symbol not in retrieved_symbols:
        gene_info = gi.gene_info(gene_symbol)
        if gene_info is not None:
            print(f"Gene info for {gene_symbol}: {gene_info}")

Gene regions for combined_CDX:

Gene regions for combined_CHS:
('CLCN4', 10156974, 10237660)

Gene regions for combined_CHB:
('PASD1', 151563674, 151676739)

Gene regions for combined_KHV:
('PASD1', 151563674, 151676739)

Gene regions for combined_JPT:





**Symbol:** **_PASD1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** CT63, CT64, OXTES1  
*PAS domain containing repressor 1*  
**Summary:** This gene encodes a protein that is thought to function as a transcription factor. The protein is a cancer-associated antigen that can stimulate autologous T-cell responses, and it is therefore considered to be a potential immunotherapeutic target for the treatment of various hematopoietic malignancies, including diffuse large B-cell lymphoma. [provided by RefSeq, May 2010].  
**Genomic position:** X:151563675-151676739 (hg38), X:150732094-150845211 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PASD1)  


 ----

**Symbol:** **_CLCN4_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** CLC4, ClC-4, ClC-4A, MRX15, MRX49, MRXSRC  
*chloride voltage-gated channel 4*  
**Summary:** The CLCN family of voltage-dependent chloride channel genes comprises nine members (CLCN1-7, Ka and Kb) which demonstrate quite diverse functional characteristics while sharing significant sequence homology. Chloride channel 4 has an evolutionary conserved CpG island and is conserved in both mouse and hamster. This gene is mapped in close proximity to APXL (Apical protein Xenopus laevis-like) and OA1 (Ocular albinism type I), which are both located on the human X chromosome at band p22.3. The physiological role of chloride channel 4 remains unknown but may contribute to the pathogenesis of neuronal disorders. Alternate splicing results in two transcript variants that encode different proteins. [provided by RefSeq, Mar 2012].  
**Genomic position:** X:10156975-10237660 (hg38), X:10125024-10205700 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=CLCN4)  


 ----

In [2]:
import pandas as pd

def read_google_sheet():
    SHEET_ID = '1JSjSLuto3jqdEnnG7JqzeC_1pUZw76n7XueVAYrUOpk'
    SHEET_NAME = 'Sheet1'
    url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
    df = pd.read_csv(url, header=1)
    return df.loc[:, [not x.startswith('Unnamed') for x in df.columns]]

def gene_list_names():
    df = read_google_sheet()
    return sorted(df.columns.tolist())

def gene_list(name):
    df = read_google_sheet()
    sr = df[name]
    return sr[~sr.isnull()]

# List of column names to be tested
columns_to_test = ['sfari_all_conf', 'intel_seiz_lang', 'intelect_disabil']

# Read the data from Google Sheets
df = read_google_sheet()

# Filter the DataFrame to include only the specified columns
df_filtered = df[columns_to_test]

# Concatenate the columns into a single Series
all_genes = pd.concat([df_filtered[col] for col in columns_to_test])

# Remove NaN values and get unique genes
unique_genes = all_genes.dropna().unique()

# Convert the array to a list for better readability
unique_genes_list = unique_genes.tolist()
unique_genes_list.sort()

# Get the number of unique genes
num_unique_genes = len(unique_genes_list)

# Print the number of unique genes and the sorted list of unique genes
print(f"Number of unique genes: {num_unique_genes}")

# Print the list of unique genes
print(unique_genes_list)


Number of unique genes: 189
['ABCB7', 'ABCD1', 'ACSL4', 'AFF2', 'AGTR2', 'AIFM1', 'ALG13', 'AMER1', 'AMMECR1', 'ANOS1', 'AP1S2', 'AR', 'ARHGEF9', 'ARSL', 'ARX', 'ATP6AP1', 'ATP6AP2', 'ATP7A', 'ATRX', 'AVPR2', 'BCAP31', 'BCOR', 'BCORL1', 'BGN', 'BRWD3', 'BTK', 'CACNA1F', 'CASK', 'CCDC22', 'CD99L2', 'CDKL5', 'CHM', 'CLCN4', 'CLIC2', 'CNKSR2', 'COX7B', 'CUL4B', 'CXorf56', 'DCX', 'DDX3X', 'DDX53', 'DKC1', 'DLG3', 'DMD', 'EBP', 'EFNB1', 'EIF2S3', 'EMD', 'FAM47A', 'FAM50A', 'FANCB', 'FGD1', 'FGF13', 'FHL1', 'FLNA', 'FMR1', 'FRMPD4', 'FTSJ1', 'GABRA3', 'GATA1', 'GDI1', 'GJB1', 'GK', 'GLA', 'GLRA2', 'GPC3', 'GPC4', 'GPR101', 'GRIA3', 'HCCS', 'HCFC1', 'HDAC6', 'HDAC8', 'HMGB3', 'HNRNPH2', 'HPRT1', 'HS6ST2', 'HSD17B10', 'HUWE1', 'IDS', 'IGBP1', 'IGSF1', 'IKBKG', 'IL1RAPL1', 'IL1RAPL2', 'IL2RG', 'IQSEC2', 'KDM5C', 'KDM6A', 'KIF4A', 'KLHL15', 'L1CAM', 'LAGE3', 'LAMP2', 'LAS1L', 'MAGT1', 'MAOA', 'MAOB', 'MBTPS2', 'MECP2', 'MED12', 'MID1', 'MID2', 'MSL3', 'MTM1', 'NAA10', 'NDP', 'NDUFA1', 'NDUFB11',

In [3]:
print(list(df['intelect_disabil']))

['ABCD1', 'ACSL4', 'AFF2', 'AIFM1', 'ALG13', 'AMER1', 'AP1S2', 'ARHGEF9', 'ARX', 'ATP6AP1', 'ATP6AP2', 'ATP7A', 'ATRX', 'AVPR2', 'BCAP31', 'BCOR', 'BCORL1', 'BRWD3', 'CASK', 'CDKL5', 'CLCN4', 'CLIC2', 'CNKSR2', 'CUL4B', 'CXorf56', 'DCX', 'DDX3X', 'DKC1', 'DLG3', 'DMD', 'EBP', 'EIF2S3', 'FAM50A', 'FANCB', 'FGD1', 'FLNA', 'FMR1', 'FRMPD4', 'FTSJ1', 'GATA1', 'GDI1', 'GK', 'GPC4', 'GRIA3', 'HCCS', 'HCFC1', 'HDAC6', 'HDAC8', 'HMGB3', 'HNRNPH2', 'HPRT1', 'HS6ST2', 'HSD17B10', 'HUWE1', 'IDS', 'IGBP1', 'IKBKG', 'IL1RAPL1', 'IQSEC2', 'KDM5C', 'KIF4A', 'KLHL15', 'L1CAM', 'LAGE3', 'LAMP2', 'LAS1L', 'MAGT1', 'MAOA', 'MBTPS2', 'MECP2', 'MED12', 'MID2', 'MSL3', 'NAA10', 'NDP', 'NEXMIF', 'NHS', 'NKAP', 'NLGN3', 'NLGN4X', 'NONO', 'NSDHL', 'OCRL', 'OFD1', 'OGT', 'OPHN1', 'OTC', 'PAK3', 'PDHA1', 'PGK1', 'PHF6', 'PHF8', 'PLP1', 'POLA1', 'PORCN', 'PQBP1', 'PRPS1', 'PTCHD1', 'RAB39B', 'RBMX', 'RLIM', 'RNF113A', 'RPL10', 'RPS6KA3', 'SHROOM4', 'SLC16A2', 'SLC6A8', 'SLC9A6', 'SLC9A7', 'SMC1A', 'SMS', 'SSR4', 

In [ ]:
from scipy.stats import fisher_exact

M = 1400 # All chrX genes
N = 200 # All autism/brain genes
n = 35 # All genes under selection 
x = 15 # Autism genes under selection

          # Selection   # No selection            
table = [[  x,           n - x          ],  # Autism genes
         [ N - x,        M - (n + N) + x]]  # Non-autism genes
table, fisher_exact(table, alternative='greater').pvalue

In [3]:
gi.gene_info("FRMPD4")

**Symbol:** **_FRMPD4_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** MRX104, PDZD10, PDZK10, XLID104  
*FERM and PDZ domain containing 4*  
**Summary:** This gene encodes a multi-domain (WW, PDZ, FERM) containing protein. Through its interaction with other proteins (such as PSD-95), it functions as a positive regulator of dendritic spine morphogenesis and density, and is required for the maintenance of excitatory synaptic transmission. [provided by RefSeq, Jan 2010].  
**Genomic position:** X:11822423-12724523 (hg38), X:12156585-12742642 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=FRMPD4)  


 ----

In [4]:
gi.gene_info("TMLHE")

**Symbol:** **_TMLHE_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** AUTSX6, BBOX2, TMLD, TMLH, TMLHED, XAP130  
*trimethyllysine hydroxylase, epsilon*  
**Summary:** This gene encodes the protein trimethyllysine dioxygenase which is the first enzyme in the carnitine biosynthesis pathway. Carnitine play an essential role in the transport of activated fatty acids across the inner mitochondrial membrane. The encoded protein converts trimethyllysine into hydroxytrimethyllysine. A pseudogene of this gene is found on chromosome X. Alternate splicing results in multiple transcript variants.[provided by RefSeq, May 2010].  
**Genomic position:** X:155489011-155719098 (hg38), HG1497_PATCH:154658583-154838412, X:154719776-154899605 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TMLHE)  


 ----

In [5]:
gi.gene_info("CDKL5")

**Symbol:** **_CDKL5_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** CFAP247, DEE2, EIEE2, ISSX, STK9  
*cyclin dependent kinase like 5*  
**Summary:** This gene is a member of Ser/Thr protein kinase family and encodes a phosphorylated protein with protein kinase activity. Mutations in this gene have been associated with X-linked infantile spasm syndrome (ISSX), also known as X-linked West syndrome, and Rett syndrome (RTT). Alternate transcriptional splice variants have been characterized. [provided by RefSeq, Jul 2008].  
**Genomic position:** X:18425583-18653629 (hg38), X:18443703-18671749 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=CDKL5)  


 ----

In [6]:
gi.gene_info("IL1RAPL1")

**Symbol:** **_IL1RAPL1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** IL-1-RAPL-1, IL-1RAPL-1, IL1R8, IL1RAPL, IL1RAPL-1, MRX10, MRX21, MRX34, OPHN4, TIGIRR-2  
*interleukin 1 receptor accessory protein like 1*  
**Summary:** The protein encoded by this gene is a member of the interleukin 1 receptor family and is similar to the interleukin 1 accessory proteins. This protein has an N-terminal signal peptide, three extracellular immunoglobulin Ig-like domains, a transmembrane domain, an intracellular Toll/IL-1R domain, and a long C-terminal tail which interacts with multiple signalling molecules. This gene is located at a region on chromosome X that is associated with a non-syndromic form of X-linked intellectual disability. Deletions and mutations in this gene were found in patients with intellectual disability. This gene is expressed at a high level in post-natal brain structures involved in the hippocampal memory system, which suggests a specialized role in the physiological processes underlying memory and learning abilities, and plays a role in synapse formation and stabilization. [provided by RefSeq, Jul 2017].  
**Genomic position:** X:28587446-29956718 (hg38), X:28605516-29974840 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=IL1RAPL1)  


 ----

In [7]:
gi.gene_info("PTCHD1-AS")

**Symbol:** **_PTCHD1-AS_**   
*PTCHD1 antisense RNA (head to head)*  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PTCHD1-AS)  


 ----

In [8]:
gi.gene_info("CLCN4")

**Symbol:** **_CLCN4_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** CLC4, ClC-4, ClC-4A, MRX15, MRX49, MRXSRC  
*chloride voltage-gated channel 4*  
**Summary:** The CLCN family of voltage-dependent chloride channel genes comprises nine members (CLCN1-7, Ka and Kb) which demonstrate quite diverse functional characteristics while sharing significant sequence homology. Chloride channel 4 has an evolutionary conserved CpG island and is conserved in both mouse and hamster. This gene is mapped in close proximity to APXL (Apical protein Xenopus laevis-like) and OA1 (Ocular albinism type I), which are both located on the human X chromosome at band p22.3. The physiological role of chloride channel 4 remains unknown but may contribute to the pathogenesis of neuronal disorders. Alternate splicing results in two transcript variants that encode different proteins. [provided by RefSeq, Mar 2012].  
**Genomic position:** X:10156975-10237660 (hg38), X:10125024-10205700 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=CLCN4)  


 ----

In [9]:
gi.gene_info("DMD")

**Symbol:** **_DMD_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** BMD, CMD3B, DXS142, DXS164, DXS206, DXS230, DXS239, DXS268, DXS269, DXS270, DXS272, MRX85  
*dystrophin*  
**Summary:** This gene spans a genomic range of greater than 2 Mb and encodes a large protein containing an N-terminal actin-binding domain and multiple spectrin repeats. The encoded protein forms a component of the dystrophin-glycoprotein complex (DGC), which bridges the inner cytoskeleton and the extracellular matrix. Deletions, duplications, and point mutations at this gene locus may cause Duchenne muscular dystrophy (DMD), Becker muscular dystrophy (BMD), or cardiomyopathy. Alternative promoter usage and alternative splicing result in numerous distinct transcript variants and protein isoforms for this gene. [provided by RefSeq, Dec 2016].  
**Genomic position:** X:31097677-33339609 (hg38), X:31115794-33357558 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=DMD)  


 ----

In [10]:
gi.gene_info("HUWE1")

**Symbol:** **_HUWE1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** ARF-BP1, HECTH9, HSPC272, Ib772, LASU1, MRXST, MULE, URE-B1, UREB1  
*HECT, UBA and WWE domain containing E3 ubiquitin protein ligase 1*  
**Summary:** This gene encodes a protein containing a C-terminal HECT (E6AP type E3 ubiquitin protein ligase) domain that functions as an E3 ubiquitin ligase. The encoded protein is required for the ubiquitination and subsequent degradation of the anti-apoptotic protein Mcl1 (myeloid cell leukemia sequence 1 (BCL2-related)). This protein also ubiquitinates the p53 tumor suppressor, core histones, and DNA polymerase beta. Mutations in this gene are associated with Turner type X-linked syndromic cognitive disability. [provided by RefSeq, Aug 2013].  
**Genomic position:** X:53532096-53686752 (hg38), HG1433_PATCH:53275144-53429752, X:53559057-53713673 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=HUWE1)  


 ----

In [11]:
gi.gene_info("IL1RAPL1")

**Symbol:** **_IL1RAPL1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** IL-1-RAPL-1, IL-1RAPL-1, IL1R8, IL1RAPL, IL1RAPL-1, MRX10, MRX21, MRX34, OPHN4, TIGIRR-2  
*interleukin 1 receptor accessory protein like 1*  
**Summary:** The protein encoded by this gene is a member of the interleukin 1 receptor family and is similar to the interleukin 1 accessory proteins. This protein has an N-terminal signal peptide, three extracellular immunoglobulin Ig-like domains, a transmembrane domain, an intracellular Toll/IL-1R domain, and a long C-terminal tail which interacts with multiple signalling molecules. This gene is located at a region on chromosome X that is associated with a non-syndromic form of X-linked intellectual disability. Deletions and mutations in this gene were found in patients with intellectual disability. This gene is expressed at a high level in post-natal brain structures involved in the hippocampal memory system, which suggests a specialized role in the physiological processes underlying memory and learning abilities, and plays a role in synapse formation and stabilization. [provided by RefSeq, Jul 2017].  
**Genomic position:** X:28587446-29956718 (hg38), X:28605516-29974840 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=IL1RAPL1)  


 ----

In [12]:
gi.gene_info("PTCHD1")

**Symbol:** **_PTCHD1_** (protein-coding) &nbsp; &nbsp; &nbsp; &nbsp; **Aliases:** AUTSX4, CXDELp22.11, DELXP22.11  
*patched domain containing 1*  
**Summary:** This gene encodes a membrane protein with a patched domain. The encoded protein is similar to Drosophila proteins which act as receptors for the morphogen sonic hedgehog. Deletions in this gene, which is located on the X chromosome, are associated with intellectual disability and autism (PMID: 21091464, PMID: 20844286). [provided by RefSeq, Aug 2011].  
**Genomic position:** X:23334849-23404374 (hg38), X:23352133-23422489 (hg19)  
[Gene card](https://www.genecards.org/cgi-bin/carddisp.pl?gene=PTCHD1)  


 ----